In [ ]:
import re # Regex (expressão regular)
!pip install unidecode
from unicodedata import normalize # utíl para tratamento de texto e compatibilidade
!pip install pandas
import pandas as pd
!pip install nltk
import nltk # Ferramentas de PLN
!pip install sklearn
import sklearn # Ferramentas de aprendizado de máquina
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
!pip install seaborn
import seaborn as sns
!pip install matplotlib
import matplotlib.pyplot as plt # Plotagem de gráficosX_train_count

print("\n" + '\033[1m' + '\033[93m' + "Importações feitas com sucesso!")

## Dados
* As seguintes CSV foram baixados em [Portuguese Tweets for Sentiment Analysis](https://www.kaggle.com/augustop/portuguese-tweets-for-sentiment-analysis) criado por [augustop](https://www.kaggle.com/augustop) 

In [ ]:
# https://www.kaggle.com/augustop/portuguese-tweets-for-sentiment-analysis

arquivo = "dados_para_treinamento.csv'

df_tweets = pd.read_csv(arquivo, sep=";", usecols=['tweet_text','sentiment'], index_col=None, header=0, dtype={"sentiment":"int16"})

df_tweets

In [ ]:
# é importante que os dados estejam proximos para não criar um 'Bias' muito grande a um tipo
df_tweets['sentiment'].value_counts()
# 1 = Positivo 🙂
# 0 = Negativo 🙁

In [ ]:
# Deixando tudo em minúsculo
df_tweets['tweet_text'] = df_tweets['tweet_text'].str.lower()

df_tweets['tweet_text']

In [ ]:
# Removendo tudo entre []
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='\[.*?\]', value='', regex=True)

df_tweets['tweet_text']

In [ ]:
# remover links
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='https?://[A-Za-z0-9./]+', value='', regex=True)
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='http?://[A-Za-z0-9./]+', value='', regex=True)
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='ftp?://[A-Za-z0-9./]+', value='', regex=True)
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='ftps?://[A-Za-z0-9./]+', value='', regex=True)
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='wwww?://[A-Za-z0-9./]+', value='', regex=True)

In [ ]:
# Removendo números
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace='\d+', value='', regex=True)

df_tweets['tweet_text']

In [ ]:
# Removendo #rashtags
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace=r'(\#\w+)', value='', regex=True)

df_tweets['tweet_text']

In [ ]:
# Removendo @Mentions
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace=r'(\@\w+)', value='', regex=True)

df_tweets['tweet_text']

In [ ]:
# Removendo a pontuação (EX: "!?.,/|#$%¨&")
df_tweets['tweet_text'] = df_tweets['tweet_text'].replace(to_replace=r'[^\w\s]', value='', regex=True)

df_tweets['tweet_text']

In [ ]:
# Removendo acentuação
df_tweets['tweet_text'] = df_tweets['tweet_text'].str.normalize('NFKD').str.encode('ASCII', errors='ignore').str.decode('UTF-8')

df_tweets['tweet_text']

In [ ]:
# Uma stopword pode ser considerada uma palavra irrelevante para a análise
nltk.download('stopwords')
# RSLP(Removedor de Sufixos da Língua Portuguesa)
nltk.download('rslp')

In [ ]:
# Criando uma lista de stopWords
stopWords = set(nltk.corpus.stopwords.words('portuguese'))

vetorizador = CountVectorizer(stop_words=stopWords)

In [ ]:
X = vetorizador.fit_transform(df_tweets.tweet_text)
y = df_tweets.sentiment

# Separa em amostras de treino -> X_train, y_train e amostras de teste -> X_test y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
modelo = naive_bayes.MultinomialNB()
modelo.fit(X_train, y_train)

In [ ]:
frases = [
    'um bom dia',
    'que coisa horrivel'
]

frases_count = vetorizador.transform(frases)

In [ ]:
teste = modelo.predict(frases_count)

In [ ]:
modelo.score(X_test,y_test)

In [ ]:
i = 1
for score in teste:
    if (score == 1):
        sentimento = "Positivo" + " " + "\U0001f642"
    elif (score == 0):
        sentimento = "Negativo" + " " + "\U0001f641"
    else:
        sentimento = "None" + " " + "\U0001f610"

    print(f"Frase N° {i} teve score {sentimento}")
    i += 1